# Metal CDR Relation Extraction 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import metal
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
print('PyTorch: ', torch.__version__)
print('MeTaL:   ', metal.__version__)
print('Python:  ', sys.version)
print('Python:  ', sys.version_info)

PyTorch:  1.0.0
MeTaL:    0.3.3
Python:   3.6.7 (default, Dec  8 2018, 17:35:14) 
[GCC 5.4.0 20160609]
Python:   sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


## Initalize CDR Dataset
To uncompress the SQLite db: ```bzip2 -d cdr.db.bz2```

In [4]:
from metal.contrib.backends.wrapper import SnorkelDataset
import os

db_conn_str   = os.path.join(os.getcwd(),"cdr.db")
candidate_def = ['ChemicalDisease', ['chemical', 'disease']]

train, dev, test = SnorkelDataset.splits(db_conn_str, 
                                         candidate_def, 
                                         max_seq_len=125)

print(f'[TRAIN] {len(train)}')
print(f'[DEV]   {len(dev)}')
print(f'[TEST]  {len(test)}')

Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/backends/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/backends/cdr.db
Connected to sqlite:////dfs/scratch0/vschen/metal/metal/contrib/backends/cdr.db
[TRAIN] 8272
[DEV]   888
[TEST]  4620


## Train End Model (Random Initalized Embeddings)

In [5]:
from metal.end_model import EndModel
from metal.modules import LSTMModule
use_cuda = torch.cuda.is_available()

lstm = LSTMModule(embed_size=50, 
                  hidden_size=100, 
                  vocab_size=train.word_dict.len(),
                  lstm_reduction='attention', 
                  dropout=0,
                  num_layers=1, 
                  freeze=False)

end_model = EndModel([200, 2], input_module=lstm, seed=123, use_cuda=use_cuda)

end_model.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
end_model.config['train_config']['validation_metric'] = 'f1'
end_model.config['train_config']['batch_size'] = 32
end_model.config['train_config']['n_epochs'] = 5


Using randomly initialized embeddings.
Embeddings shape = (9946, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'

Network architecture:
Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(9946, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
  (1): Linear(in_features=200, out_features=2, bias=True)
)



In [6]:
end_model.train_model(train, dev_data=dev)

Using GPU...



Saving model at iteration 0 with best score 0.580
[E:0]	Train Loss: 0.564	Dev f1: 0.580



[E:1]	Train Loss: 0.328	Dev f1: 0.549



Saving model at iteration 2 with best score 0.594
[E:2]	Train Loss: 0.191	Dev f1: 0.594



[E:3]	Train Loss: 0.118	Dev f1: 0.571



[E:4]	Train Loss: 0.092	Dev f1: 0.589
Restoring best model from iteration 2 with score 0.594
Finished Training
F1: 0.594
        y=1    y=2   
 l=1    183    137   
 l=2    113    455   


In [7]:
score = end_model.score(test, metric=['precision', 'recall', 'f1'])

Precision: 0.517
Recall: 0.660
F1: 0.580
        y=1    y=2   
 l=1    993    927   
 l=2    512   2188   


## Train End Model (Pretrained Embeddings)

Download [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip):
`wget http://nlp.stanford.edu/data/glove.6B.zip \
&& mkdir -p glove.6B \
&& unzip glove.6B.zip -d glove.6B \
&& rm glove.6B.zip`

In [8]:
import string
import numpy as np

import torch.nn.init as init

class EmbeddingLoader(object):
    """
    Simple text file embedding loader. Words with GloVe and FastText.
    """
    def __init__(self, fpath, fmt='text', dim=None, normalize=True):
        assert os.path.exists(fpath)
        self.fpath = fpath
        self.dim = dim
        self.fmt = fmt
        # infer dimension
        if not self.dim:
            header = open(self.fpath, "rU").readline().strip().split(' ')
            self.dim = len(header) - 1 if len(header) != 2 else int(header[-1])

        self.vocab, self.vectors = zip(*[(w,vec) for w,vec in self._read()])
        self.vocab = {w:i for i,w in enumerate(self.vocab)}
        self.vectors = np.vstack(self.vectors)
        if normalize:
            self.vectors = (self.vectors.T / np.linalg.norm(self.vectors, axis=1, ord=2)).T

    def _read(self):
        start = 0 if self.fmt == "text" else 1
        for i, line in enumerate(open(self.fpath, "rU")):
            if i < start:
                continue
            line = line.rstrip().split(' ')
            vec = np.array([float(x) for x in line[1:]])
            if len(vec) != self.dim:
                errors += [line[0]]
                continue
            yield (line[0], vec)
            

def load_embeddings(vocab, embeddings):
    """
    Load pretrained embeddings
    """
    def get_word_match(w, word_dict):
        if w in word_dict:
            return word_dict[w]
        elif w.lower() in word_dict:
            return word_dict[w.lower()]
        elif w.strip(string.punctuation) in word_dict:
            return word_dict[w.strip(string.punctuation)]
        elif w.strip(string.punctuation).lower() in word_dict:
            return word_dict[w.strip(string.punctuation).lower()]
        else:
            return -1

    num_words = vocab.len()
    emb_dim   = embeddings.vectors.shape[1]
    vecs      = init.xavier_normal_(torch.empty(num_words, emb_dim))
    vecs[0]   = torch.zeros(emb_dim)

    n = 0
    for w in vocab.d:
        idx = get_word_match(w, embeddings.vocab)
        if idx == -1:
            continue
        i = vocab.lookup(w)
        vecs[i] = torch.FloatTensor(embeddings.vectors[idx])
        n += 1

    print("Loaded {:2.1f}% ({}/{}) pretrained embeddings".format(float(n) / vocab.len() * 100.0, n, vocab.len() ))
    return vecs         

In [9]:
emb_path  = "glove.6B/glove.6B.50d.txt"
embs  = EmbeddingLoader(emb_path, fmt='text')

/dfs/scratch0/vschen/snorkel-pytorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: 'U' mode is deprecated
/dfs/scratch0/vschen/snorkel-pytorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: 'U' mode is deprecated


In [10]:
from metal.contrib.backends.wrapper import SnorkelDataset

db_conn_str   = "cdr.db"
candidate_def = ['ChemicalDisease', ['chemical', 'disease']]

train, dev, test = SnorkelDataset.splits(db_conn_str, 
                                         candidate_def, 
                                         pretrained_word_dict=embs.vocab, 
                                         max_seq_len=125)

print(f'[TRAIN] {len(train)}')
print(f'[DEV]   {len(dev)}')
print(f'[TEST]  {len(test)}')

Connected to sqlite:///cdr.db
Connected to sqlite:///cdr.db
Connected to sqlite:///cdr.db
[TRAIN] 8272
[DEV]   888
[TEST]  4620


### Initalize pretrained embedding matrix

In [11]:
wembs = load_embeddings(train.word_dict, embs)

Loaded 79.9% (9116/11406) pretrained embeddings


In [12]:
from metal.end_model import EndModel
from metal.modules import LSTMModule
use_cuda = torch.cuda.is_available()

lstm = LSTMModule(embed_size=50, 
                  hidden_size=100, 
                  embeddings=wembs,
                  lstm_reduction='attention', 
                  dropout=0, 
                  num_layers=1, 
                  freeze=False)

end_model = EndModel([200, 2], input_module=lstm, seed=123, use_cuda=use_cuda)

end_model.config['train_config']['optimizer_config']['optimizer_common']['lr'] = 0.01
end_model.config['train_config']['validation_metric'] = 'f1'
end_model.config['train_config']['batch_size'] = 32
end_model.config['train_config']['n_epochs'] = 5

Using pretrained embeddings.
Embeddings shape = (11406, 50)
The embeddings are NOT FROZEN
Using lstm_reduction = 'attention'

Network architecture:
Sequential(
  (0): Sequential(
    (0): LSTMModule(
      (embeddings): Embedding(11406, 50)
      (lstm): LSTM(50, 100, batch_first=True, bidirectional=True)
    )
    (1): ReLU()
  )
  (1): Linear(in_features=200, out_features=2, bias=True)
)



In [13]:
end_model.train_model(train, dev_data=dev)

Using GPU...



Saving model at iteration 0 with best score 0.599
[E:0]	Train Loss: 0.533	Dev f1: 0.599



[E:1]	Train Loss: 0.293	Dev f1: 0.568



[E:2]	Train Loss: 0.174	Dev f1: 0.560



[E:3]	Train Loss: 0.119	Dev f1: 0.585



[E:4]	Train Loss: 0.078	Dev f1: 0.575
Restoring best model from iteration 0 with score 0.599
Finished Training
F1: 0.599
        y=1    y=2   
 l=1    191    151   
 l=2    105    441   


In [14]:
score = end_model.score(test, metric=['precision', 'recall', 'f1'])

Precision: 0.526
Recall: 0.658
F1: 0.585
        y=1    y=2   
 l=1    990    892   
 l=2    515   2223   
